In [ ]:
!pip install mediapipe-model-maker
!pip install tensorflow==2.15.1
!pip install tf-models-official==2.15

In [1]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
# Link Google Drive with Colab (provide necessery permissions) / Optional ⚠️

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/asl_alphabet_train.zip

Streaming output truncated to the last 5000 lines.
 extracting: asl_alphabet_train/Y/Y19.jpg  
 extracting: asl_alphabet_train/Y/Y190.jpg  
 extracting: asl_alphabet_train/Y/Y1900.jpg  
 extracting: asl_alphabet_train/Y/Y1901.jpg  
 extracting: asl_alphabet_train/Y/Y1902.jpg  
 extracting: asl_alphabet_train/Y/Y1903.jpg  
 extracting: asl_alphabet_train/Y/Y1904.jpg  
 extracting: asl_alphabet_train/Y/Y1905.jpg  
 extracting: asl_alphabet_train/Y/Y1906.jpg  
 extracting: asl_alphabet_train/Y/Y1907.jpg  
 extracting: asl_alphabet_train/Y/Y1908.jpg  
 extracting: asl_alphabet_train/Y/Y1909.jpg  
 extracting: asl_alphabet_train/Y/Y191.jpg  
 extracting: asl_alphabet_train/Y/Y1910.jpg  
 extracting: asl_alphabet_train/Y/Y1911.jpg  
 extracting: asl_alphabet_train/Y/Y1912.jpg  
 extracting: asl_alphabet_train/Y/Y1913.jpg  
 extracting: asl_alphabet_train/Y/Y1914.jpg  
 extracting: asl_alphabet_train/Y/Y1915.jpg  
 extracting: asl_alphabet_train/Y/Y1916.jpg  
 extracting: asl_alphabet_train/Y

In [3]:
dataset_path = "asl_alphabet_train"
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

asl_alphabet_train
['U', 'F', 'G', 'X', 'space', 'I', 'C', 'D', 'V', 'O', 'M', 'E', 'J', 'B', 'H', 'S', 'Q', 'T', 'N', 'W', 'Y', 'A', 'del', 'K', 'R', 'none', 'P', 'Z', 'L']


In [5]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Training-data


In [6]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 29)                3741      
 out (Dense)                                                     
                                                             

In [7]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

5971/5971 [==============================] - 36s 2ms/step - loss: 0.1990 - categorical_accuracy: 0.9350
Test loss:0.19900478422641754, Test accuracy:0.9350192546844482


In [8]:
# Export the model
model.export_model()

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite


In [9]:
!ls exported_model
files.download('exported_model/gesture_recognizer.task')

best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>